In [4]:
import numpy as np
import open3d as o3d
from visual import mstorePCR

view_count = 16
point_data = "blocks"

views = [o3d.io.read_point_cloud(f"./data/pcs/{point_data}/point_cloud_{i}.pcd") for i in range(view_count)]

point_cloud_obj = mstorePCR(views)

o3d.io.write_point_cloud(f"./data/pcs/{point_data}/mstore.pcd", point_cloud_obj)

True

In [6]:
pcd = o3d.io.read_point_cloud(f"./data/pcs/{point_data}/mstore.pcd")
pcd = pcd.voxel_down_sample(voxel_size=.005)
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.025, max_nn=20))

o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [7]:
step = 2*np.pi/view_count

for i in range(view_count):
    pcd = o3d.io.read_point_cloud(f"./data/pcs/{point_data}/point_cloud_{i}.pcd")
    pcd = pcd.voxel_down_sample(voxel_size=.005)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Center point cloud
    min_coor = np.array([
        min([p[0] for p in pcd.points]),
        min([p[1] for p in pcd.points]),
        min([p[2] for p in pcd.points])
    ])

    max_coor = np.array([
        max([p[0] for p in pcd.points]),
        max([p[1] for p in pcd.points]),
        max([p[2] for p in pcd.points])
    ])

    midpoint = (max_coor+min_coor)/2

    for i in range(len(pcd.points)):
        pcd.points[i] -= midpoint
        tmp = pcd.points[i][1]
        pcd.points[i][1] = pcd.points[i][2]
        pcd.points[i][2] = tmp

    # Rotate point cloud
    angle = i*step
    rotation_matrix = np.array([
        [np.cos(angle), 0, np.sin(angle)],
        [0, 1, 0],
        [-np.sin(angle), 0, np.cos(angle)]
    ])

    pcd.rotate(rotation_matrix)

    # Calculate and display normals
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.025, max_nn=20))
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)

KeyboardInterrupt: 